### This notebook is taken as a reference from [here](https://www.kaggle.com/ryanholbrook/create-your-first-submission)

In [3]:
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning.ex_tpu import *
step_1.check()

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-yp6s8as3
  Running command git clone -q https://github.com/Kaggle/learntools.git /tmp/pip-req-build-yp6s8as3
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=205145 sha256=89b95463aeed51bc499afbc8810da1979469bc5fc94e0035110164f0bc1f2a9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-cgmulpn3/wheels/dd/d7/6b/0fc758f52767fd281d6dceded6757c6cb5bb90ccd2dbb1de9f
Successfully built learntools


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Setup complete.</span>

### Loading helper functions

In [4]:
## let's load the data from the utility script
from petal_helper import *

import tensorflow as tf

Tensorflow version 2.2.0
Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8
Dataset: 12753 training images, 3712 validation images, 7382 unlabeled test images


### Distribution strategy

In [5]:
# Lets learn the distribution startegy for the TPU's. 
# Each TPU has 8 cores (each core is like a GPU in itself)
# We need to tell tensorflow on how to make use of this TPU by a distribution strategy

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)
    


Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


### Loading the dataset 

In [6]:
## Loading the data from the competition

# when using TPUs datasets are often serialized into TFRecords.
# This is a convenient format to feed to e ach of the TPU cores
# petal_helper utility script will load the TFRecords and create a data pipeline 
# to use with our model 


ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training : ", ds_train)
print("Validation : ", ds_valid)
print("Testing : ", ds_test)

print("type : ", type(ds_test))
# These are tf.data.Dataset objects. You can think about the dataset in Tensorflow as a stream of data records


Training :  <PrefetchDataset shapes: ((None, 512, 512, 3), (None,)), types: (tf.float32, tf.int32)>
Validation :  <PrefetchDataset shapes: ((None, 512, 512, 3), (None,)), types: (tf.float32, tf.int32)>
Testing :  <PrefetchDataset shapes: ((None, 512, 512, 3), (None,)), types: (tf.float32, tf.string)>
type :  <class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


### Defining the model

In [21]:
# We'll use Transfer learning where we use an already built pre-trained model
# and we can retrain a part of the models neural network to get a head-start on our new dataset

# The distribution strategy we created earlier contains a context manager, strategy.scope. 
# This context manager tells TensorFlow how to divide the work of training among the eight TPU cores. 
# When using TensorFlow with a TPU, it's important to define your model in a strategy.scope() context.


with strategy.scope():
    pretrained_model = tf.keras.applications.VGG16(
    weights = "imagenet",
    input_shape = [*IMAGE_SIZE, 3],
    include_top = False)
    
    
    pretrained_model.trainable = False
    
    model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(CLASSES), activation='softmax')])
    
    model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['sparse_categorical_accuracy'])
    

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 16, 16, 512)       14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 104)               53352     
Total params: 14,768,040
Trainable params: 53,352
Non-trainable params: 14,714,688
_________________________________________________________________


### Training the model 

In [22]:
# Define the batch size. This will be 16 with TPU off and 128 (=16*8) with TPU on
BATCH_SIZE = 16*strategy.num_replicas_in_sync

# Defining the epochs
EPOCHS = 10
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

history = model.fit(
    ds_train,
    validation_data = ds_valid,
    epochs = EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH
    
)

Epoch 1/10
99/99 [==============================] - 26s 261ms/step - sparse_categorical_accuracy: 0.0855 - loss: 4.1094 - val_sparse_categorical_accuracy: 0.1226 - val_loss: 3.8940
Epoch 2/10
99/99 [==============================] - 18s 183ms/step - sparse_categorical_accuracy: 0.1652 - loss: 3.7685 - val_sparse_categorical_accuracy: 0.1956 - val_loss: 3.6464
Epoch 3/10
99/99 [==============================] - 17s 175ms/step - sparse_categorical_accuracy: 0.2113 - loss: 3.5490 - val_sparse_categorical_accuracy: 0.2276 - val_loss: 3.4547
Epoch 4/10
99/99 [==============================] - 17s 168ms/step - sparse_categorical_accuracy: 0.2370 - loss: 3.3678 - val_sparse_categorical_accuracy: 0.2538 - val_loss: 3.2994
Epoch 5/10
99/99 [==============================] - 17s 169ms/step - sparse_categorical_accuracy: 0.2677 - loss: 3.2097 - val_sparse_categorical_accuracy: 0.2845 - val_loss: 3.1640
Epoch 6/10
99/99 [==============================] - 17s 171ms/step - sparse_categorical_accurac

In [ ]:
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    history.history['sparse_categorical_accuracy'],
    history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

### Predictions

In [15]:
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

Computing predictions...
[ 67  28 103 ...  48  53  53]


In [23]:
## Let us generate submission.csv file.

print("generating submission.csv file ")


# Get image ids from test set and convert to unicode
test_ids_ds =  test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')


# Write submission.csv file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

generating submission.csv file 


In [24]:
# Look at the first few predictions
!head submission.csv


id,label
252d840db,67
1c4736dea,28
c37a6f3e9,103
00e4f514e,103
59d1b6146,53
8d808a07b,53
aeb67eefb,103
53cfc6586,48
aaa580243,67
